## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving cookie_cats.csv to cookie_cats.csv


In [2]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

p1 = 0.19
p2 = 0.20
alpha = 0.05
power = 0.8

effect_size = proportion_effectsize(p1, p2)

analysis = NormalIndPower()
n_per_group = analysis.solve_power(
    effect_size=effect_size,
    power=power,
    alpha=alpha,
    ratio=1)

n_per_group

24637.863136236123

2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
df = pd.read_csv("cookie_cats.csv")

retention_7_means = df.groupby("version")["retention_7"].mean()

retention_7_means

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


In [4]:
# H₀: утримання на 7-й день у gate_40 не гірше, ніж у gate_30
# H₁: версія gate_30 дає краще утримання на 7-й день, ніж gate_40
# це односторонній тест на різницю пропорцій
# згідно зі спостереженням з даних маємо:
# gate_30: 19.02%
# gate_40: 18.20%
# тобто у вибірці утримання на 7-й день вище у gate_30
# тобто нульову гіпотезу відкидаємо

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [5]:
# gate_30 - це control
# gate_40 - це treatment

# ми питаємо чи дає якась версія кращий retention, що означає що це двосторонній тест:
# H₀: p(gate_30)​=p(gate_40)​
# H₁: p(gate_30)=!p(gate_40)​

In [6]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control = df[df["version"] == "gate_30"]["retention_7"]
treatment = df[df["version"] == "gate_40"]["retention_7"]

successes = np.array([control.sum(), treatment.sum()])
nobs = np.array([len(control), len(treatment)])

In [7]:
z_stat, p_value = proportions_ztest(
    count=successes,
    nobs=nobs,
    alternative="two-sided")

In [8]:
ci_control = proportion_confint(
    count=control.sum(),
    nobs=len(control),
    alpha=0.05,
    method="normal")

ci_treatment = proportion_confint(
    count=treatment.sum(),
    nobs=len(treatment),
    alpha=0.05,
    method="normal")

In [9]:
print(f"z statistic: {z_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Довірчий інтервал 95% для групи control: [{ci_control[0]:.4f}, {ci_control[1]:.4f}]")
print(f"Довірчий інтервал 95% для групи treatment: [{ci_treatment[0]:.4f}, {ci_treatment[1]:.4f}]")

z statistic: 3.1644
p-value: 0.0016
Довірчий інтервал 95% для групи control: [0.1866, 0.1938]
Довірчий інтервал 95% для групи treatment: [0.1785, 0.1855]


In [10]:
# робимо висновки:
# 1. різниця є статистично значущою (p-value = 0.0016 < 0.05,
# тобто ймовірність отримати таку різницю випадково = 0.16%)
# відхиляємо нульову гіпотезу
# 2. верхня межа treatment = 0.1855, нижня межа control = 0.1866
# 0.1855 < 0.1866
# довірчі інтервали не перетинаються (це означає, що різниця не є випадковою)
# перенесення воріт з рівня 30 на рівень 40 погіршило утримання користувачів на 7й день
# версія gate_30 статистично значущо краща за показником retention_7.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [11]:
# ми перевіряємо наступні показники:
# version: gate_30 / gate_40
# retention_7: 0 (не повернувся) / 1 (повернувся)

# H₀: версія гри незалежна від утримання гравця на 7-й день
# (тобто між version і retention_7 немає зв'язку)
# H₁: між версією гри та утриманням гравця на 7-й день існує залежність

In [12]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(
    df["version"],
    df["retention_7"])

contingency_table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [13]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

In [14]:
print(f"Chi-square statistic: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")

Chi-square statistic: 9.9591
p-value: 0.0016


In [15]:
# p-value < 0.05, отже нульову гіпотезу відхиляємо.
# маємо статистично значущі докази того, що версія гри
# дійсно пов'язана з утриманням гравця на 7-й день.